# Coding attention mechanism

## Attending to different parts of the input with self-Attention

### A simple self-attention mechanism without trainable weights.


In [2]:
import torch

inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],  # Your     (x^1)
        [0.55, 0.87, 0.66],  # journey  (x^2)
        [0.57, 0.85, 0.64],  # starts   (x^3)
        [0.22, 0.58, 0.33],  # with     (x^4)
        [0.77, 0.25, 0.10],  # one      (x^6)
        [0.05, 0.80, 0.55],  # step     (x^7)
    ]
)

In [12]:
input_query = inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [13]:
input_1 = inputs[0]
input_1

tensor([0.4300, 0.1500, 0.8900])

In [14]:
0.55*0.43 + 0.87*0.15 + 0.66*0.89

0.9544

In [15]:

# compute the attention score

torch.dot(input_query, input_1)

tensor(0.9544)

In [16]:
res = 0.

for idx, ele in enumerate(inputs[0]):
    print(inputs[0][idx])

tensor(0.4300)
tensor(0.1500)
tensor(0.8900)


In [21]:
res = 0.
i = 0

for idx, ele in enumerate(inputs[i]):
    # res += inputs[i][idx] *  input_query[idx]
res += inputs[i][idx] *  torch.dot(inputs[i], input_query)


In [23]:
inputs.shape

torch.Size([6, 3])

In [24]:
# using for all inputs
# using attn_score_2 cause we are trying to find wrt to token 2

query  = inputs[1]
attn_score_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_score_2[i] = torch.dot(query, x_i)

print(attn_score_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [25]:
# now from scores lets create the attention weights

# step 1 we normalize the attention scores to add up to 1

attn_weights_temp = attn_score_2/attn_score_2.sum()
print(attn_weights_temp)
print(attn_weights_temp.sum())




tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
tensor(1.0000)


In [36]:
def softmax_naive(x):
    return torch.exp(x)/torch.exp(x).sum(dim=0)

softmax_naive(attn_score_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [44]:
attn_weights_2 = torch.softmax(attn_score_2, dim=0)  # stable and optimized stable function

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


# Compute the context vector

![Attention Mechanism Context Vector](./docs/nb_images/cal_norm_attention_weights.png)

The context vector is a weighted sum of values, where weights come from the attention mechanism. It's computed by multiplying each input vector with its corresponding attention weight and summing the results.


In [39]:
print(attn_weights_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [45]:
# attn_scores
i = 1
query= inputs[i]

context_vec_2 = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * inputs[i]

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


# Simple self attention mechanism without trainable weights


In [47]:
# lets find attn scores for all the inputs

attn_scores = torch.empty(inputs.shape[0], inputs.shape[0])

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [52]:
attn_scores = inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_scores)
print(f"\nNormalised attention weights:")
print(attn_weights)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

Normalised attention weights:
tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [50]:
context_vec =

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [57]:
print(attn_weights@inputs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


# Implementing self-attention with trainable weights

## computing the attention mechanism step by step